In [20]:
import ast
import os
from urllib.parse import urlparse
import pprint
import pandas as pd
import json
from random import randint

In [21]:
class Extractor:
    def __init__(self):
        urlList = []

    def readDataIntoList(self, filename):
        if filename == "SVM_C5.txt":
            with open(filename, "r") as fp:
                x = json.load(fp)
            return x
        
        with open(filename, "r") as fp:
            data = fp.read()
            x = ast.literal_eval(data)
            
        return x
    
    def jsondump(self, js, file):
        with open(file, 'w') as outfile:
            json.dump(js, outfile, indent=4)
        
    def fetchDomain(selfself, url):
        parsed_uri = urlparse(url)
        result = '{uri.scheme}://{uri.netloc}/'.format(uri=parsed_uri)
        #print(result)
        return result

    def clean(self, urls, minscore, maxscore, file):
        finalList = []

        for item in urls:
            d = dict()
            if item[1][1] == "NaN":
                continue
            d["url"] = item[1][0]
            d["score"] = item[1][1]
            d["domain"] = self.fetchDomain(item[1][0])
            finalList.append(d)
        # pprint.pprint(finalList, indent=4)
        uniqueDomains = dict()
        uniqueList = []
        for item in finalList:
            if item["score"]>=minscore and item["score"]<maxscore:
                if item["domain"] in uniqueDomains:
                    uniqueDomains[item["domain"]]["counts"] += 1
                    uniqueDomains[item["domain"]]["urls"].append(item["url"])
                else:
                    uniqueDomains[item["domain"]] = dict()
                    uniqueDomains[item["domain"]]["counts"] = 1
                    uniqueDomains[item["domain"]]["urls"] = [item["url"]]
        # pprint.pprint(uniqueDomains)   
        print(len(uniqueDomains))
        self.jsondump(uniqueDomains, "Final_Data/cleaned_data/"+file[:2]+"_top_all.json")    
        return uniqueDomains
    
    def extract(self, data, top):
        totalUnique = len(data) #total unique URLs
        finalList = []
        if top<totalUnique:
            for item in data:
                index = randint(0,len(data[item])-1)   
                finalList.append(data[item]["urls"][index]) 
        else:
            for item in data:
                i=0
                maxEach = int(top/totalUnique)
                while i<maxEach and i<len(data[item]["urls"]):
                    finalList.append(data[item]["urls"][i])
                    i+=1
            if len(finalList)<top:
                dif = top - len(finalList) # more urls needed
                i = 0
                for item in data:
                    if i>= dif:
                        break
                    count = 0
                    while True: 
                        index = randint(0,len(data[item]["urls"])-1)    
                        if data[item]["urls"][index] not in finalList:
                            finalList.append(data[item]["urls"][index])
                            i+=1
                            break
                        else:
                            if count>len(data[item]["urls"]): # exceeds 
                                break
                            count += 1
        return finalList         
            

In [22]:
if __name__ == "__main__":
    ex = Extractor()
    path = "Final_Data/"
    all_list = []
    for file in os.listdir(path):
        if file[-5:] == "5.txt" or file[:6] == "Cosine": # 5 class urls
            print(file)
            if file[:2] == "SV" or file[:6] == "Cosine":
                with open(path+file, "r") as fp:
                    data = json.load(fp)
            else:
                data = ex.readDataIntoList(path+file)
#           file = "Final_Data/NB_C5.txt"
            top = 40 # total number of URLs to extract
            try:
                if file[:2] == "RF" or file[:2] == "NN":
                    minscore = 3
                    maxscore = 5
                elif file[:6] == "Cosine":
                    minscore = 0.5
                    maxscore = 1
                else:
                    minscore = 4
                    maxscore = 5
                print("--------------\nMax\n--------------\n")
                cleaned_data = ex.clean(data, minscore, maxscore, file)
                final_list = ex.extract(cleaned_data, top)
#                 pprint.pprint(final_list)
                all_list += final_list
            except:
                print("ERROR")
            try:
                print("--------------\nMin\n--------------\n")
                if file[:2] == "RF" or file[:2] == "NN":
                    minscore = 2
                    maxscore = 3
                elif file[:6] == "Cosine":
                    minscore = 0.3
                    maxscore = 0.5
                else:
                    minscore = 3
                    maxscore = 4
                cleaned_data = ex.clean(data, minscore, maxscore, file)
                final_list = ex.extract(cleaned_data, top)
#                 pprint.pprint(final_list)
                all_list += final_list
            except:
                print("ERROR")
        
#     pprint.pprint(all_list, indent=4)
    print(len(all_list))
    with open("Final_Data/final_list.txt", "a") as fp:
        for item in all_list:
            fp.write(item+"\n")
    

RF_C5.txt
--------------
Max
--------------

11
--------------
Min
--------------

12
NB_C5.txt
--------------
Max
--------------

12
--------------
Min
--------------

11
NN_Fixed_C5.txt
--------------
Max
--------------

14
--------------
Min
--------------

14
Cosine_Results.txt
--------------
Max
--------------

11
--------------
Min
--------------

14
SVM_C5.txt
--------------
Max
--------------

12
--------------
Min
--------------

14
400
